# Fast selection on object id

Little demo of finding an object by its ID in hipscat, if you don't know the radec.

Start by looking at the hipscat-library index catalog.

In [2]:
from hipscat.catalog.index.index_catalog import IndexCatalog
import pandas as pd

catalog_index = IndexCatalog.read_from_hipscat("/data3/epyc/data3/hipscat/test_catalogs/agn_sdss_id2")
catalog_index

These are a handful of AGNs. Let's check the index catalog, and see if we can reduce this list of IDs into just those pixels of interest.

In [3]:
import pandas as pd

pd.read_parquet("/data3/epyc/data3/hipscat/test_catalogs/agn_sdss_id/index/part.0.parquet").query("Npix == 5104").head()


,_hipscat_index,Norder,Dir,Npix
SDSS_NAME,,,,
000000.33+310325.3,5746857072813670400,5,0,5104
000000.62+311944.3,5746873713899339776,5,0,5104
000000.72+305758.5,5746820779341774848,5,0,5104
000001.63+303109.6,5746653227206574080,5,0,5104
000003.53+303413.3,5746659613743251456,5,0,5104


In [4]:

ids = ["000000.15+353104.2", "000000.33+310325.3",
      "000003.61+344738.9", "000005.56+334037.5",
      "000000.72+305758.5", "000003.53+303413.3"]

In [5]:
%%time
partitions = catalog_index.loc_partitions(ids)
partitions

CPU times: user 201 ms, sys: 42.5 ms, total: 243 ms
Wall time: 168 ms


array([Order: 5, Pixel: 5104, Order: 5, Pixel: 5107], dtype=object)

Now let's try that with lsdb, and fetch the full rows of these objects.

In [6]:
%%time
import lsdb

agns = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/dr16q_constant")

CPU times: user 2.23 s, sys: 354 ms, total: 2.58 s
Wall time: 3.19 s


In [7]:
%%time
index_search_df = agns.index_search(ids, catalog_index).compute()
index_search_df

CPU times: user 597 ms, sys: 131 ms, total: 728 ms
Wall time: 714 ms


,SDSS_NAME,PLATE,MJD,FIBERID,RA,DEC,OBJID,IF_BOSS_SDSS,Z_DR16Q,SOURCE_Z_DR16Q,...,LOGMBH_MGII_ERR,LOGMBH_CIV,LOGMBH_CIV_ERR,LOGMBH,LOGMBH_ERR,LOGLEDD_RATIO,LOGLEDD_RATIO_ERR,Norder,Dir,Npix
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
5746659613743251456,000003.53+303413.3,7134,56566,594,0.014710,30.570384,7134-56566-0594,BOSS,2.165000,VI,...,-1.000000,8.490171,0.180997,8.490171,0.180997,-0.864171,0.181369,5,0,5104
5746820779341774848,000000.72+305758.5,7749,58073,360,0.003016,30.966277,7749-58073-0360,BOSS,1.622798,PIPE,...,0.322852,0.000000,-1.000000,9.108047,0.322852,-1.411452,0.323024,5,0,5104
5746857072813670400,000000.33+310325.3,7749,58073,660,0.001415,31.057048,7749-58073-0660,BOSS,2.035491,PIPE,...,-1.000000,8.985320,0.163913,8.985320,0.163913,-1.388920,0.164950,5,0,5104
5750189105279401984,000005.56+334037.5,7750,58402,178,0.023199,33.677090,7750-58402-0178,BOSS,2.374468,PIPE,...,0.065516,0.000000,-1.000000,9.399690,0.065516,-0.778144,0.065528,5,0,5107
5750881315637428224,000003.61+344738.9,7750,58402,186,0.015076,34.794156,7750-58402-0186,BOSS,1.954585,PIPE,...,0.392049,8.085483,0.164185,7.684127,0.392049,-0.045617,0.392439,5,0,5107
5751092381348790272,000000.15+353104.2,7750,58402,802,0.000629,35.517841,7750-58402-0802,BOSS,0.845435,PIPE,...,0.036066,0.000000,-1.000000,9.073521,0.036066,-0.948884,0.036082,5,0,5107


In [11]:
%%time
full_scan_flags = agns._ddf["SDSS_NAME"].isin(ids).compute()
full_scan_df = agns._ddf[full_scan_flags].compute()
full_scan_df

/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/dataframe/core.py:5074: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  meta = self._meta[_extract_meta(key)]
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key 

/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series

/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series

/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series

/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series

...

/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series

/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series

/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))
/astro/users/mmd11/.conda/envs/hipscatenv/lib/python3.10/site-packages/dask/core.py:127: UserWarning: Boolean Series

CPU times: user 6min 14s, sys: 2min 8s, total: 8min 22s
Wall time: 4min 12s


,SDSS_NAME,PLATE,MJD,FIBERID,RA,DEC,OBJID,IF_BOSS_SDSS,Z_DR16Q,SOURCE_Z_DR16Q,...,LOGMBH_MGII_ERR,LOGMBH_CIV,LOGMBH_CIV_ERR,LOGMBH,LOGMBH_ERR,LOGLEDD_RATIO,LOGLEDD_RATIO_ERR,Norder,Dir,Npix
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
5746659613743251456,000003.53+303413.3,7134,56566,594,0.014710,30.570384,7134-56566-0594,BOSS,2.165000,VI,...,-1.000000,8.490171,0.180997,8.490171,0.180997,-0.864171,0.181369,5,0,5104
5746820779341774848,000000.72+305758.5,7749,58073,360,0.003016,30.966277,7749-58073-0360,BOSS,1.622798,PIPE,...,0.322852,0.000000,-1.000000,9.108047,0.322852,-1.411452,0.323024,5,0,5104
5746857072813670400,000000.33+310325.3,7749,58073,660,0.001415,31.057048,7749-58073-0660,BOSS,2.035491,PIPE,...,-1.000000,8.985320,0.163913,8.985320,0.163913,-1.388920,0.164950,5,0,5104
5750189105279401984,000005.56+334037.5,7750,58402,178,0.023199,33.677090,7750-58402-0178,BOSS,2.374468,PIPE,...,0.065516,0.000000,-1.000000,9.399690,0.065516,-0.778144,0.065528,5,0,5107
5750881315637428224,000003.61+344738.9,7750,58402,186,0.015076,34.794156,7750-58402-0186,BOSS,1.954585,PIPE,...,0.392049,8.085483,0.164185,7.684127,0.392049,-0.045617,0.392439,5,0,5107
5751092381348790272,000000.15+353104.2,7750,58402,802,0.000629,35.517841,7750-58402-0802,BOSS,0.845435,PIPE,...,0.036066,0.000000,-1.000000,9.073521,0.036066,-0.948884,0.036082,5,0,5107
